Парсинг русских сайтов

In [1]:
# Загрузка нужных библиотек

import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import random
from datetime import datetime
from datetime import timedelta

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os
import re
import telegram

Собираем 500 ссылок на новости с сайта https://knife.media/category/news/
Принцип нумерации страниц - https://knife.media/category/news/page/2/

In [263]:
# Собираем 500 ссылок на новости с сайта https://knife.media/category/news/
# Принцип нумерации страниц - https://knife.media/category/news/page/2/

def get_all_links_R1(pages):
    all_refs = []
    for i in range(1, pages+1):
        if i == 1:
            URL = 'https://knife.media/category/news/'
        else:
            URL = 'https://knife.media/category/news/page/' + str(i) + '/'
        res = requests.get(URL)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text)
        news_blocks = soup.find_all('a', class_='widget-news__content-link')
        all_refs += list(map(lambda x: x.get('href'), news_blocks))
        print(i)
    return all_refs

all_links = get_all_links_R1(25)
all_links

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


['https://knife.media/comedy-wildlife-photo/',
 'https://knife.media/antibacterial-figs/',
 'https://knife.media/sintospichki/',
 'https://knife.media/insects-migration/',
 'https://knife.media/gpt-mouse/',
 'https://knife.media/lektsii-znanie/',
 'https://knife.media/max-alcohol/',
 'https://knife.media/bat-penis/',
 'https://knife.media/navel-of-the-world/',
 'https://knife.media/handsome-career/',
 'https://knife.media/explosive-gold/',
 'https://knife.media/bill-gates-list/',
 'https://knife.media/hej-hej/',
 'https://knife.media/cosmic-erection/',
 'https://knife.media/trans-emperor/',
 'https://knife.media/brain-battle/',
 'https://knife.media/tampon-for-men/',
 'https://knife.media/antarctic-cucumber/',
 'https://knife.media/middle-age-job/',
 'https://knife.media/baby-shark/',
 'https://knife.media/drunk-birds/',
 'https://knife.media/penguin-couples/',
 'https://knife.media/witcher-cookbook/',
 'https://knife.media/rest-in-peace/',
 'https://knife.media/move-for-information/',

In [6]:
len(all_links)

500

Собираем даты, заголовки и тексты новостей

In [12]:
all_news = pd.DataFrame()
prom_loop = 1
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.1 * np.random.randint(1, 10))
    # Определение времени новости
    date = pd.to_datetime(soup.find('time').get('datetime')).date()
    #print(date)
    
    # Определение названия новости
    title = str(soup.find('em')).replace('<em>', '').replace('</em>', '')
    #print(title)
    
    # Определение текста новости
    prom_text = str(soup.find('div', class_='entry-content').find_all('p'))
    text = ''
    include_letter = False
    for letter in prom_text:
        if letter == '>': 
            include_letter = True
            continue
        if letter == '<': 
            include_letter = False
            continue
        if include_letter: 
            if letter.isdigit() or letter.isalpha() or letter.isalnum() or (letter in '.,;:"?!-%^&*()\=+/[]{}«»—– '): text += letter
    #print(text)
    
    row = {
        'date': date, 'is_Politic': 0, 'is_Science': 0, 'is_Funny': 0,
        'title_text': title + text, 'site': 'knife.media', 'url': link, 'title': title, 'text': text
    }
    all_news = pd.concat([all_news, pd.DataFrame([row])])
    
    print(prom_loop)
    prom_loop += 1

all_news = all_news.reset_index().drop('index', axis=1)
all_news

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,date,title,text
0,2023-11-25,топ-5 лекций последней недели ноября от общест...,Общество «Знание» ежедневно проводит лекции на...
1,2023-11-24,топ-5 лекций последней недели ноября от общест...,Минздрав Японии подготовил проект рекомендаций...
2,2023-11-24,топ-5 лекций последней недели ноября от общест...,Ученые изучили видео спаривающихся летучих мыш...
3,2023-11-23,трейлер новой части мультика «Три богатыря»,Российская студия анимационного кино «Мельница...
4,2023-11-23,симпатичные мужчины строят более успешную карьеру,Исследователи из Норвегии и Польши изучили дан...
...,...,...,...
495,2023-08-03,"послушайте, как звучит «мерцающая» звезда",Астрономы создали модели движения энергии от я...
496,2023-08-03,топ-5 лекций последней недели ноября от общест...,Вцентре дизайна Artplay 1августа открылась мул...
497,2023-08-03,Он может приклеиваться и отклеиваться по команде,"Японские ученые разработали новый клей, которы..."
498,2023-08-03,"младенцы дольше смотрят на те картины, которые...","Исследователи обнаружили, что картины, которые..."


In [62]:
all_news.to_csv('news_russia_1')

In [52]:
# Максималная длина "заголовка + текста новости" с ПЕРВОГО новостного сайта
max([len(all_news.loc[x, 'title_text']) for x in range(500)])

19549

In [85]:
# Максималная длина "заголовка + текста новости" со ВТОРОГО новостного сайта
max([len(all_news2.loc[x, 'title_text']) for x in range(524)])

23174

Собираем 500 ссылок на новости с сайта https://naked-science.ru/article
Принцип нумерации страниц - https://naked-science.ru/article/page/2

Собираем новости со второго сайта

In [2]:
# Определяем функцию, которая будет инициировать соединение с сайтом для парсинга
def create_driver(url):
   options = webdriver.ChromeOptions()
   options.add_argument("start-maximized")
   options.add_argument("--headless")  # работа без открытия окна браузера
   options.add_argument(
       "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
       "Chrome/105.0.0.0 Safari/537.36")
   options.add_argument("--disable-blink-features=AutomationControlled")

   driver = webdriver.Chrome(options=options)

   driver.get(url)

   return driver  # возвращаем драйвер

In [4]:
# Собираем 500 ссылок на новости с сайта https://naked-science.ru/article
# Принцип нумерации страниц - https://naked-science.ru/article/page/2

def get_all_links_R2(pages):
    all_refs = []
    for page in range(1, pages+1):
        if page == 1:
            URL = 'https://naked-science.ru/article/'
        else:
            URL = 'https://naked-science.ru/article/page/' + str(page) + '/'
        
        driver = create_driver(URL)  # создаем драйвер

        all_links = driver.find_elements(By.CLASS_NAME, "animate-custom")  # получаем все элементы с классом animate-custom
        hrefs = []  # пока пустой массив для самих ссылок

        for link in all_links:
            hrefs.append(link.get_attribute("href"))  # получаем у каждой ссылки атрибут href
        # Слишком много получилось ссылок - раз в 5 больше. Будем удалять лишние
            
        # Сначала удаляем ссылки None
        for i in reversed(range(len(hrefs))):
            if hrefs[i] == None: hrefs.remove(None) 
        
        # Прежде всего удаляем ссылки на другие сайлы - ведь мы собираем новости на этом
        # Исполузую реверсивный генератор чисел, чтобы при удалении элемента по индексу остальные элементы оставались на местах
        for i in reversed(range(len(hrefs))):
            if hrefs[i].find('naked-science.ru/article') == -1: hrefs.pop(i)  
            
        # Нужные ссылки должны быть длиенные = количество символов '/' должно быть 5 и больше
        # Снова реверсивный генератор чисел по той же причине
        for i in reversed(range(len(hrefs))):
            prom_number = 0
            for symbol in hrefs[i]: 
                if symbol == '/': prom_number += 1
            if prom_number < 5: hrefs.pop(i)  
            
        # Как оказалось, некоторые ссылки задваиваются - удаляем дупликаты
        hrefs = list(set(hrefs))
        
        all_refs += hrefs
        print(f'Страница {page}, к-во новых ссылок {len(hrefs)}')
    
    return all_refs

all_links2 = get_all_links_R2(25)
all_links2 = list(set(all_links2))
print(f'Количество собранных сслыок {len(all_links2)}')
all_links2

Страница 1, к-во новых ссылок 26
Страница 2, к-во новых ссылок 32
Страница 3, к-во новых ссылок 30
Страница 4, к-во новых ссылок 33
Страница 5, к-во новых ссылок 33
Страница 6, к-во новых ссылок 32
Страница 7, к-во новых ссылок 33
Страница 8, к-во новых ссылок 32
Страница 9, к-во новых ссылок 33
Страница 10, к-во новых ссылок 33
Страница 11, к-во новых ссылок 33
Страница 12, к-во новых ссылок 33
Страница 13, к-во новых ссылок 33
Страница 14, к-во новых ссылок 33
Страница 15, к-во новых ссылок 33
Страница 16, к-во новых ссылок 33
Страница 17, к-во новых ссылок 33
Страница 18, к-во новых ссылок 33
Страница 19, к-во новых ссылок 33
Страница 20, к-во новых ссылок 33
Страница 21, к-во новых ссылок 33
Страница 22, к-во новых ссылок 33
Страница 23, к-во новых ссылок 33
Страница 24, к-во новых ссылок 33
Страница 25, к-во новых ссылок 33


['https://naked-science.ru/article/hi-tech/openai-evil',
 'https://naked-science.ru/article/physics/fiziki-otkryli',
 'https://naked-science.ru/article/anthropology/neolithic-shipwreck',
 'https://naked-science.ru/article/astronomy/second-highest-energy',
 'https://naked-science.ru/article/physics/eksperimentatory-vpervye',
 'https://naked-science.ru/article/interview/v-nebo-smotryat',
 'https://naked-science.ru/article/medicine/orehov-dlya-muzhskoi-fert',
 'https://naked-science.ru/article/medicine/ochistiteli-vozduha',
 'https://naked-science.ru/article/column/poluchdov-obogashheniya-a',
 'https://naked-science.ru/article/column/v-mgppna-psihiku-i-kreati',
 'https://naked-science.ru/article/hi-tech/roskosh-iz-vozduha-kak-i',
 'https://naked-science.ru/article/psy/trudogoliki-chuvstvuyut',
 'https://naked-science.ru/article/column/v-tyya-klyuchevyh-fraz-iz',
 'https://naked-science.ru/article/psy/privlekatehe-chem-zhensko',
 'https://naked-science.ru/article/psy/v-svoem-budushhem-fina

In [8]:
df2 = pd.DataFrame(all_links2)
df2

,0
0,https://naked-science.ru/article/hi-tech/opena...
1,https://naked-science.ru/article/column/razrab...
2,https://naked-science.ru/article/anthropology/...
3,https://naked-science.ru/article/medicine/skol...
4,https://naked-science.ru/article/psy/zhertvy-s...
...,...
519,https://naked-science.ru/article/column/fiziki...
520,https://naked-science.ru/article/medicine/kros...
521,https://naked-science.ru/article/medicine/opas...
522,https://naked-science.ru/article/astronomy/bol...


In [9]:
df2.to_excel('links_from_2_site.xlsx')

In [81]:
# Собирает даты, заголовки и тексты отобранных 500 новостей
all_news2 = pd.DataFrame()

for link in range(df2.shape[0]):
    # xbnftv cnhfybwe
    driver = webdriver.Chrome()
    driver.get(df2['Links'][link])
    # Заголовко новости
    title = driver.title 
    # Дата новости
    my_date = pd.to_datetime(driver.find_element(by=By.CLASS_NAME, value="echo_date").get_attribute("data-published")).date()
    # Получаем текст новости
    all_texts = driver.find_elements(by=By.TAG_NAME, value="p")
    flag = False
    my_text = ''
    for prom_text in all_texts:
        if flag and prom_text.text == '': break
        my_text = ' '.join( [my_text, prom_text.text] )
        if prom_text.text != '': flag = True
    my_text = my_text.strip()

    driver.implicitly_wait(0.5)
    driver.quit()
    
    row = {
        'date': my_date, 'is_Politic': 0, 'is_Science': 0, 'is_Funny': 0,
        'title_text': title + my_text, 'site': 'knife.media', 'url': df2['Links'][link], 'title': title, 'text': my_text
    }
    all_news2 = pd.concat([all_news2, pd.DataFrame([row])])
    
    print(link)
    
all_news2 = all_news2.reset_index().drop('index', axis=1)
all_news2





0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

NameError: name 'all_news' is not defined

In [83]:
all_news2 = all_news2.reset_index().drop('index', axis=1)
all_news2

,date,is_Politic,is_Science,is_Funny,title_text,site,url,title,text
0,2023-11-23,0,0,0,Увольнение Сэма Альтмана попытались связать с ...,knife.media,https://naked-science.ru/article/hi-tech/opena...,Увольнение Сэма Альтмана попытались связать с ...,"Анонимные источники Reuters утверждают, что пе..."
1,2023-10-24,0,0,0,Разработана методика изготовления сверхпроводн...,knife.media,https://naked-science.ru/article/column/razrab...,Разработана методика изготовления сверхпроводн...,Ученые из МФТИ и ИРЭ имени В. А. Котельникова ...
2,2023-09-26,0,0,0,"Мужчине, жившему 1600 лет назад, поставили диа...",knife.media,https://naked-science.ru/article/anthropology/...,"Мужчине, жившему 1600 лет назад, поставили диа...","Ученые применили современные методы, такие как..."
3,2023-09-29,0,0,0,"Ученые узнали, сколько шагов по лестнице снижа...",knife.media,https://naked-science.ru/article/medicine/skol...,"Ученые узнали, сколько шагов по лестнице снижа...",Знаменитое «правило десяти тысяч шагов» может ...
4,2023-11-10,0,0,0,«Смартфонная зависимость» оказалась разборчиво...,knife.media,https://naked-science.ru/article/psy/zhertvy-s...,«Смартфонная зависимость» оказалась разборчиво...,"Оказалось, число людей, проводящих за смартфон..."
...,...,...,...,...,...,...,...,...,...
519,2023-09-26,0,0,0,Физики нашли лучший способ приготовления ферри...,knife.media,https://naked-science.ru/article/column/fiziki...,Физики нашли лучший способ приготовления ферритов,Ученые из МФТИ и Южно-Уральского государственн...
520,2023-10-26,0,0,0,Крошечный прыгающий паук «свил гнездо» в челов...,knife.media,https://naked-science.ru/article/medicine/kros...,Крошечный прыгающий паук «свил гнездо» в челов...,Тайваньские врачи из города Тайнань рассказали...
521,2023-11-02,0,0,0,Этикетки с предупреждающими надписями об опасн...,knife.media,https://naked-science.ru/article/medicine/opas...,Этикетки с предупреждающими надписями об опасн...,Чрезмерная любовь к красному мясу приводит к н...
522,2023-09-28,0,0,0,Большинство лавовых планет оказались похожими ...,knife.media,https://naked-science.ru/article/astronomy/bol...,Большинство лавовых планет оказались похожими ...,"Лавовые миры — экзопланеты, напоминающие горяч..."


In [84]:
all_news2.to_excel('Good_news_russia_2.xlsx')

Финальные штрихи по разделению 

In [76]:
# Читаем таблицу
df1 = pd.read_excel('Good_news_russia_1.xlsx')
# Исправляем неточность - нет разделителя между названием и текстом статьи
df1['title_text'] = df1['title'] + '. ' + df1['text']
# Упорядочиваем таблицу по дате и перенумеровываем индекс
df1 = df1.sort_values(by='date', ascending=False).reset_index().drop('index', axis=1)
# Отбираем последние 100 новостей для сверки BERT с работой человека
df1_final_test = df1[0:100]
# Выбираем оставшиеся новости
df1_prom = df1[100:]
# Из них выбираем 90% для обучения алгоритма
df1_training_start = df1_prom.sample(frac=0.9)
# А так же отбираем остальные 10% для донастройки алгоритма
df1_prom_list = df1_training_start.index.to_list()
df1_training_check = df1_prom[~df1_prom.index.isin(df1_prom_list)]

df2 = pd.read_excel('Good_news_russia_2.xlsx')
df2['title_text'] = df2['title'] + '. ' + df2['text']
df2 = df2.sort_values(by='date', ascending=False).reset_index().drop('index', axis=1)
df2_final_test = df2[0:100]
df2_prom = df2[100:]
df2_training_start = df2_prom.sample(frac=0.9)
df2_prom_list = df2_training_start.index.to_list()
df2_training_check = df2_prom[~df2_prom.index.isin(df2_prom_list)]

# Объединяем новости с двух сайтов в один DataFrame для алгоритма
df_final_test = df1_final_test.append(df2_final_test)
df_training_start = df1_training_start.append(df2_training_start)
df_training_check = df1_training_check.append(df2_training_check)

C:\Users\Ferrick\AppData\Local\Temp\ipykernel_28608\4045503112.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final_test = df1_final_test.append(df2_final_test)
C:\Users\Ferrick\AppData\Local\Temp\ipykernel_28608\4045503112.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_training_start = df1_training_start.append(df2_training_start)
C:\Users\Ferrick\AppData\Local\Temp\ipykernel_28608\4045503112.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_training_check = df1_training_check.append(df2_training_check)
